## Feature extraction for training the svm model to identify trends

In [1]:
!pip install textblob



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\shira\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### 1. Sentiment Analysis

In [9]:
import pandas as pd
from textblob import TextBlob

merged_df = pd.read_csv('final.csv')

# A helper function to calculate sentiment polarity
def calculate_polarity(text):
    return TextBlob(text).sentiment.polarity


# Ensure all entries in 'Review' column are strings
merged_df['Review'] = merged_df['Review'].astype(str)

# Then, you can proceed with the sentiment calculation
merged_df['Sentiment'] = merged_df['Review'].apply(calculate_polarity)


In [10]:
merged_df.head(10)

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Review,Sentiment
0,38,Low-Fat Berry Blue Frozen Dessert,Dancer,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,5,"Yummy, yummy, yummy! I am a big fan of fruit a...",0.500000
1,38,Low-Fat Berry Blue Frozen Dessert,Dancer,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,4,very good!! the directions were alittle weird ...,0.250000
2,38,Low-Fat Berry Blue Frozen Dessert,Dancer,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,4,This does taste great! I think the recipe need...,0.279167
3,38,Low-Fat Berry Blue Frozen Dessert,Dancer,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,4,Tasty and refreshing! I love the creamy flavor...,0.562500
4,39,Biryani,elly9812,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,3,I have an Indian friend who made this dish for...,0.050000
5,40,Best Lemonade,Stephen Little,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,5,My favourite lemonade recipe is essentially th...,0.234722
6,40,Best Lemonade,Stephen Little,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,1,complicated recipe but cool.,-0.075000
7,40,Best Lemonade,Stephen Little,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,4,"The taste was excellent, and it really wasn't ...",0.378182
8,40,Best Lemonade,Stephen Little,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,5,I made this with limes instead of lemosn and i...,0.937500
9,40,Best Lemonade,Stephen Little,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,4,Really nice lemonade. Used a potato peeler to ...,0.440000


In [11]:
# Define a dictionary for the aggregation functions
agg_funcs = {
    'Name': 'first',  # takes the first name
    'AuthorName_x': 'first',  # takes the first author name
    'DatePublished': 'first',  # takes the first date
    'Description': 'first',  # takes the first description
    'Rating': 'mean',  # calculates the mean rating
    'Sentiment': 'mean',  # calculates the mean sentiment
    'Review': ' '.join  # concatenates the reviews
}

# Group the DataFrame by 'RecipeId' and apply the aggregation functions
merged_df_grouped = merged_df.groupby('RecipeId').agg(agg_funcs).reset_index()


In [13]:
# Load Excel table into a DataFrame
instructions_df = pd.read_csv('recipes.csv')  

# Merge the instructions into the grouped DataFrame
merged_df_grouped = pd.merge(merged_df_grouped, instructions_df[['RecipeId', 'RecipeInstructions']], on='RecipeId', how='left')


In [16]:
import numpy as np

merged_df_grouped['Followers'] = np.random.randint(10, 51, size=len(merged_df_grouped))

In [20]:
merged_df_grouped['NormalizedRating'] = (merged_df_grouped['Rating'] - merged_df_grouped.groupby('Followers')['Rating'].transform('mean')) / merged_df_grouped.groupby('Followers')['Rating'].transform('std')
merged_df_grouped

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating
0,38,Low-Fat Berry Blue Frozen Dessert,Dancer,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,4.250000,0.397917,"Yummy, yummy, yummy! I am a big fan of fruit a...","c(""Toss 2 cups berries with sugar."", ""Let stan...",20,-0.077061
1,39,Biryani,elly9812,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,3.000000,0.050000,I have an Indian friend who made this dish for...,"c(""Soak saffron in warm milk for 5 minutes and...",49,-1.394981
2,40,Best Lemonade,Stephen Little,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,4.333333,0.397198,My favourite lemonade recipe is essentially th...,"c(""Into a 1 quart Jar with tight fitting lid, ...",38,0.017379
3,41,Carina's Tofu-Vegetable Kebabs,Cyclopz,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,4.500000,0.516250,I thought this was very GOOD! ANYONE who is af...,"c(""Drain the tofu, carefully squeezing out exc...",30,0.168116
4,42,Cabbage Soup,Duckie067,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,2.666667,0.300929,This is a very good soup. We served ours warm...,"c(""Mix everything together and bring to a boil...",21,-1.684604
...,...,...,...,...,...,...,...,...,...,...,...
271669,540899,Butter Pecan Banana Muffins,jarp4,2020-09-01T20:59:00Z,Make and share this Butter Pecan Banana Muffin...,5.000000,0.337500,We had these for breakfast this morning. They ...,"c(""Heat oven to 375 degrees. Coat a standard 1...",21,0.654851
271670,541001,Homemade Adobo Seasoning,Wing-Man,2020-09-24T16:53:00Z,Make and share this Homemade Adobo Seasoning r...,0.000000,-0.071667,"This appears to be a dry Mexican style adobo, ...","c(""Mix all ingredients together."", ""Store in a...",22,-4.501297
271671,541030,Everything but the Bagel Seasoning (Trader Joe...,duonyte,2020-10-02T20:48:00Z,This is a copycat of this popular seasoning mi...,5.000000,0.106667,Made this mix to go with bagels and cream chee...,"c(""Mix everything together and transfer to an ...",12,0.663360
271672,541195,The Most Refreshing Lemonade,YummyFood510,2020-11-09T16:13:00Z,Lemonade is so refreshing. You won't believe h...,5.000000,0.683333,This lemonade recipe was AMAZING! So fresh and...,"c(""In a saucepan, add the sugar and 1 cup cold...",46,0.670697


### Data Filtering

In [19]:
# Calculate the 90th percentile for 'Sentiment' and 'NormalizedRating'
sentiment_threshold = merged_df_grouped['Sentiment'].quantile(0.9)
normalized_rating_threshold = merged_df_grouped['NormalizedRating'].quantile(0.9)

# Filter the DataFrame to include only the rows with 'Sentiment' and 'NormalizedRating' in the top 10 percent
top_recipes = merged_df_grouped[(merged_df_grouped['Sentiment'] >= sentiment_threshold) & 
                                (merged_df_grouped['NormalizedRating'] >= normalized_rating_threshold)]


top_recipes

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating
101,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,"c(""Melt 15g of the butter in a pan."", ""Add the...",45,0.670846
103,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,"c(""Beat all but eggs and marshmallows for two ...",46,0.670697
130,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,"c(""In a bowl, dissolve candies in water (keep ...",49,0.683363
186,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,"c(""Freshen salt cod by soaking it overnight in...",17,0.678428
198,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,"c(""Dress fresh crabs or thaw frozen crabs. Ri...",46,0.670697
...,...,...,...,...,...,...,...,...,...,...,...
271481,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,"c(""Preheat oven to 425 degrees F. Line a bakin...",18,0.677847
271567,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...","c(""Cut chicken or tofu into long strips ( if g...",50,0.675840
271607,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,"c(""Cut potatoes into thick slices. Heat the bu...",46,0.670697
271664,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,"c(""In a blender or food processor, blend the c...",49,0.683363


### Data Cleaning

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
# Create a new DataFrame for the cleaned post descriptions
df_clean = top_recipes.copy()  
df_clean['RecipeInstructions'] = df_clean['RecipeInstructions'].str.lower()  # convert to lower case
df_clean['RecipeInstructions'] = df_clean['RecipeInstructions'].str.replace('[^\w\s]', '')  # remove punctuation

df_clean

C:\Users\shira\AppData\Local\Temp\ipykernel_7612\2066673125.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean['RecipeInstructions'] = df_clean['RecipeInstructions'].str.replace('[^\w\s]', '')  # remove punctuation


,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating
101,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,cmelt 15g of the butter in a pan add the onion...,45,0.670846
103,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,cbeat all but eggs and marshmallows for two mi...,46,0.670697
130,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,cin a bowl dissolve candies in water keep warm...,49,0.683363
186,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,cfreshen salt cod by soaking it overnight in c...,17,0.678428
198,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,cdress fresh crabs or thaw frozen crabs rinse...,46,0.670697
...,...,...,...,...,...,...,...,...,...,...,...
271481,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847
271567,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...",ccut chicken or tofu into long strips if gril...,50,0.675840
271607,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,ccut potatoes into thick slices heat the butte...,46,0.670697
271664,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,cin a blender or food processor blend the cant...,49,0.683363


###  Loads a pre-trained Named Entity Recognition model using Spacy.

In [22]:
import spacy
# Load the pre-trained NER model
nlp = spacy.load('./models/')

In [23]:
# Define a function to extract food names
def extract_food_names(text):
    text = text.replace('-', ' ')
    doc = nlp(text)
    food_names = [ent.text for ent in doc.ents if ent.label_ == 'FOOD']
    return food_names

# Apply the function to the 'RecipeInstructions' column
df_clean['Ingredients'] = df_clean['RecipeInstructions'].apply(extract_food_names)
df_clean

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating,Ingredients
101,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,cmelt 15g of the butter in a pan add the onion...,45,0.670846,[salt and pepper to taste and cook]
103,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,cbeat all but eggs and marshmallows for two mi...,46,0.670697,[]
130,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,cin a bowl dissolve candies in water keep warm...,49,0.683363,[reminisce magazine by lisa andis\n]
186,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,cfreshen salt cod by soaking it overnight in c...,17,0.678428,[corn or hominy heated milk and freshened salt...
198,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,cdress fresh crabs or thaw frozen crabs rinse...,46,0.670697,"[heavier breading let crabs sit, cayenne peppe..."
...,...,...,...,...,...,...,...,...,...,...,...,...
271481,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847,"[pico de gallo, chopped cilantro]"
271567,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...",ccut chicken or tofu into long strips if gril...,50,0.675840,[]
271607,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,ccut potatoes into thick slices heat the butte...,46,0.670697,[]
271664,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,cin a blender or food processor blend the cant...,49,0.683363,"[cin, cocktail shaker, the cantaloupe puree te..."


In [26]:
# Assuming other_df is your other DataFrame
other_df = pd.read_csv('recipes.csv')

# Merge on RecipeId
df_clean = pd.merge(df_clean, other_df[['RecipeId', 'RecipeIngredientParts']], on='RecipeId', how='left')

# Drop the original 'Ingredients' column
df_clean = df_clean.drop(columns='Ingredients')

# Rename the 'RecipeIngredientParts' column to 'Ingredients'
df_clean = df_clean.rename(columns={'RecipeIngredientParts': 'Ingredients'})
df_clean

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating,Ingredients
0,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,cmelt 15g of the butter in a pan add the onion...,45,0.670846,"c(""butter"", ""onions"", ""mushrooms"", ""dry white ..."
1,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,cbeat all but eggs and marshmallows for two mi...,46,0.670697,"c(""flour"", ""white sugar"", ""baking soda"", ""butt..."
2,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,cin a bowl dissolve candies in water keep warm...,49,0.683363,"c(""boiling water"", ""lemon gelatin"", ""applesauc..."
3,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,cfreshen salt cod by soaking it overnight in c...,17,0.678428,"c(""salt cod fish"", ""fat salt pork"", ""onion"", ""..."
4,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,cdress fresh crabs or thaw frozen crabs rinse...,46,0.670697,"c(""soft shelled crabs"", ""milk"", ""flour"", ""eggs..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4328,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847,"c(""unsalted butter"", ""red onions"", ""garlic clo..."
4329,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...",ccut chicken or tofu into long strips if gril...,50,0.675840,"c(""boneless skinless chicken"", ""lemongrass"", ""..."
4330,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,ccut potatoes into thick slices heat the butte...,46,0.670697,"c(""potatoes"", ""butter"", ""onion"", ""bacon"")"
4331,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,cin a blender or food processor blend the cant...,49,0.683363,"c(""cantaloupe"", ""lime juice"", ""sugar"", ""coarse..."


In [27]:
df_clean['Ingredients'] = df_clean['Ingredients'].str.lower()  # convert to lower case
df_clean['Ingredients'] = df_clean['Ingredients'].str.replace('[^\w\s]', '')  # remove punctuation

df_clean

C:\Users\shira\AppData\Local\Temp\ipykernel_7612\2134171091.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean['Ingredients'] = df_clean['Ingredients'].str.replace('[^\w\s]', '')  # remove punctuation


,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating,Ingredients
0,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,cmelt 15g of the butter in a pan add the onion...,45,0.670846,cbutter onions mushrooms dry white wine tomato...
1,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,cbeat all but eggs and marshmallows for two mi...,46,0.670697,cflour white sugar baking soda buttermilk butt...
2,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,cin a bowl dissolve candies in water keep warm...,49,0.683363,cboiling water lemon gelatin applesauce cream ...
3,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,cfreshen salt cod by soaking it overnight in c...,17,0.678428,csalt cod fish fat salt pork onion raw potatoe...
4,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,cdress fresh crabs or thaw frozen crabs rinse...,46,0.670697,csoft shelled crabs milk flour eggs salt
...,...,...,...,...,...,...,...,...,...,...,...,...
4328,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847,cunsalted butter red onions garlic cloves red ...
4329,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...",ccut chicken or tofu into long strips if gril...,50,0.675840,cboneless skinless chicken lemongrass onions s...
4330,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,ccut potatoes into thick slices heat the butte...,46,0.670697,cpotatoes butter onion bacon
4331,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,cin a blender or food processor blend the cant...,49,0.683363,ccantaloupe lime juice sugar coarse salt


In [30]:
df_clean = df_clean.drop(columns='FoodNames')
df_clean

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating,Ingredients
0,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,cmelt 15g of the butter in a pan add the onion...,45,0.670846,cbutter onions mushrooms dry white wine tomato...
1,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,cbeat all but eggs and marshmallows for two mi...,46,0.670697,cflour white sugar baking soda buttermilk butt...
2,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,cin a bowl dissolve candies in water keep warm...,49,0.683363,cboiling water lemon gelatin applesauce cream ...
3,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,cfreshen salt cod by soaking it overnight in c...,17,0.678428,csalt cod fish fat salt pork onion raw potatoe...
4,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,cdress fresh crabs or thaw frozen crabs rinse...,46,0.670697,csoft shelled crabs milk flour eggs salt
...,...,...,...,...,...,...,...,...,...,...,...,...
4328,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847,cunsalted butter red onions garlic cloves red ...
4329,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...",ccut chicken or tofu into long strips if gril...,50,0.675840,cboneless skinless chicken lemongrass onions s...
4330,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,ccut potatoes into thick slices heat the butte...,46,0.670697,cpotatoes butter onion bacon
4331,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,cin a blender or food processor blend the cant...,49,0.683363,ccantaloupe lime juice sugar coarse salt


In [33]:

# Split each string into a list of words
df_clean['Ingredients'] = df_clean['Ingredients'].str.split()

df_clean

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating,Ingredients
0,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,cmelt 15g of the butter in a pan add the onion...,45,0.670846,"[cbutter, onions, mushrooms, dry, white, wine,..."
1,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,cbeat all but eggs and marshmallows for two mi...,46,0.670697,"[cflour, white, sugar, baking, soda, buttermil..."
2,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,cin a bowl dissolve candies in water keep warm...,49,0.683363,"[cboiling, water, lemon, gelatin, applesauce, ..."
3,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,cfreshen salt cod by soaking it overnight in c...,17,0.678428,"[csalt, cod, fish, fat, salt, pork, onion, raw..."
4,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,cdress fresh crabs or thaw frozen crabs rinse...,46,0.670697,"[csoft, shelled, crabs, milk, flour, eggs, salt]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4328,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847,"[cunsalted, butter, red, onions, garlic, clove..."
4329,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...",ccut chicken or tofu into long strips if gril...,50,0.675840,"[cboneless, skinless, chicken, lemongrass, oni..."
4330,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,ccut potatoes into thick slices heat the butte...,46,0.670697,"[cpotatoes, butter, onion, bacon]"
4331,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,cin a blender or food processor blend the cant...,49,0.683363,"[ccantaloupe, lime, juice, sugar, coarse, salt]"


### Ingredient Frequency Calculation


In [34]:
from collections import Counter

# Combine all ingredient lists into one big list
all_ingredients = [ingredient for sublist in df_clean['Ingredients'] for ingredient in sublist]

# Count the frequency of each unique ingredient
ingredient_counter = Counter(all_ingredients)

# Display the frequency of each ingredient
for ingredient, count in ingredient_counter.items():
    print(f"{ingredient}: {count}")


cbutter: 315
onions: 295
mushrooms: 133
dry: 229
white: 442
wine: 359
tomato: 224
puree: 32
boneless: 67
chicken: 344
breasts: 100
cflour: 117
sugar: 2039
baking: 632
soda: 290
buttermilk: 90
butter: 1252
margarine: 218
vanilla: 734
eggs: 616
miniature: 30
marshmallows: 39
cboiling: 12
water: 746
lemon: 620
gelatin: 47
applesauce: 32
cream: 801
cheese: 1431
mayonnaise: 164
pecans: 156
celery: 209
csalt: 27
cod: 8
fish: 32
fat: 27
salt: 1834
pork: 73
onion: 888
raw: 11
potatoes: 153
carrot: 102
boiling: 40
pepper: 1531
canned: 46
corn: 163
niblet: 3
milk: 671
csoft: 1
shelled: 1
crabs: 1
flour: 1049
walnuts: 137
extract: 298
halfandhalf: 76
cinnamon: 462
nutmeg: 185
cpartskim: 2
ricotta: 31
mozzarella: 94
ham: 55
oregano: 192
leaves: 216
cmango: 3
strawberries: 72
nectarines: 2
fresh: 1236
raspberries: 31
frozen: 107
unsweetened: 31
ginger: 224
lime: 198
juice: 649
cider: 103
vinegar: 366
red: 525
flakes: 127
allpurpose: 290
confectioners: 81
cool: 56
whip: 66
instant: 100
chocolate: 17

### Total Mentions Calculation

In [35]:
# Create a dictionary from the Counter object
ingredient_count_dict = dict(ingredient_counter)

# Define a function to calculate total mentions for each recipe
def calculate_mentions(ingredients):
    return sum(ingredient_count_dict.get(ingredient, 0) for ingredient in ingredients)

# Apply the function to the 'Ingredients' column to create the 'Number_of_Mentions' column
df_clean['Number_of_Mentions'] = df_clean['Ingredients'].apply(calculate_mentions)
df_clean

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating,Ingredients,Number_of_Mentions
0,174,Chicken Supreme with Mushrooms,Doreen Randal,1999-09-13T03:21:00Z,Make and share this Chicken Supreme with Mushr...,5.0,0.679167,excellent recipe!!! It was suggested in the Au...,cmelt 15g of the butter in a pan add the onion...,45,0.670846,"[cbutter, onions, mushrooms, dry, white, wine,...",2540
1,176,Cola Cake,george nuckols,1999-08-22T04:41:00Z,Make and share this Cola Cake recipe from Food...,5.0,0.875000,This is a good one!,cbeat all but eggs and marshmallows for two mi...,46,0.670697,"[cflour, white, sugar, baking, soda, buttermil...",6499
2,207,Cinnamon Apple Salad,Ron Joyce Ripple S,1999-08-22T04:34:00Z,Make and share this Cinnamon Apple Salad recip...,5.0,0.675000,We went to a cookout and my wife made this to ...,cin a bowl dissolve candies in water keep warm...,49,0.683363,"[cboiling, water, lemon, gelatin, applesauce, ...",4218
3,283,Salt Cod Chowder,Julesong,1999-09-10T19:07:00Z,Salt cod is used for cooking in Scandinavian a...,5.0,0.543750,This is a terrific recipe. We loved the bits o...,cfreshen salt cod by soaking it overnight in c...,17,0.678428,"[csalt, cod, fish, fat, salt, pork, onion, raw...",8200
4,297,Virginia Fried Soft Shell Crabs,Crab Place,1999-09-10T04:34:00Z,Make and share this Virginia Fried Soft Shell ...,5.0,0.609524,Yum is all I can say. Splendid recipe! I added...,cdress fresh crabs or thaw frozen crabs rinse...,46,0.670697,"[csoft, shelled, crabs, milk, flour, eggs, salt]",4173
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4328,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847,"[cunsalted, butter, red, onions, garlic, clove...",19590
4329,538765,Vietnamese Vermicelli Bowl With Lemongrass,gailanng,2019-05-08T19:14:00Z,Something that excites me...but not so much my...,5.0,0.625000,"First time to make this today, used shrimp, de...",ccut chicken or tofu into long strips if gril...,50,0.675840,"[cboneless, skinless, chicken, lemongrass, oni...",16935
4330,539115,Dutch Fried Potatoes (Gebakken Aardappelen),Chef PotPie,2019-07-25T00:17:00Z,Delicious side dish made from leftover boiled ...,5.0,0.557771,Mr picky was very happy for breakfast Made the...,ccut potatoes into thick slices heat the butte...,46,0.670697,"[cpotatoes, butter, onion, bacon]",2317
4331,540716,Cantaloupe Margarita,Chef PotPie,2020-07-24T13:32:00Z,I've been trying all kinds of margaritas this ...,5.0,0.537500,Well hello! We loved this! My only change was ...,cin a blender or food processor blend the cant...,49,0.683363,"[ccantaloupe, lime, juice, sugar, coarse, salt]",4758


In [36]:
top_percentile = df_clean["Number_of_Mentions"].quantile(0.90)  
top_percentile_df = df_clean[df_clean["Number_of_Mentions"] > top_percentile]
top_percentile_df

,RecipeId,Name,AuthorName_x,DatePublished,Description,Rating,Sentiment,Review,RecipeInstructions,Followers,NormalizedRating,Ingredients,Number_of_Mentions
14,3119,Seafood Lasagna With Brandied-Basil Cream Sauce,HELEN PEAGRAM_40951,1999-10-04T18:59:00Z,Make and share this Seafood Lasagna With Brand...,5.0,0.575298,We were looking for a recipe for Christmas Eve...,cpreheat oven to 350 degrees this recipe shoul...,26,0.676068,"[cricotta, cheese, parmesan, cheese, provolone...",18574
79,14297,Spaghetti Sauce,Kurt Godden,2001-11-14T14:30:00Z,"This recipe is from my wife's family, and is a...",5.0,0.653750,I loved this. Being a vegetarian I left out t...,cpour crushed tomatoes tomato sauce and tomato...,18,0.677847,"[ctomatoes, tomato, sauce, tomato, paste, grou...",13465
100,16740,Quesadillas with Three Cheeses,Elyse,2002-01-05T14:23:00Z,Make and share this Quesadillas with Three Che...,5.0,0.566667,I loved these. I was unsure about cooking the...,cin small nonstick skillet heat 34 teaspoon oi...,17,0.678428,"[colive, oil, onion, red, bell, pepper, green,...",17457
130,20259,Armenian Shish Kebab,Mark Marcarian,2002-02-20T15:42:00Z,Make and share this Armenian Shish Kebab recip...,5.0,0.628506,This was awesome! Excellent and simple meal. ...,cplace lamb in a large container with a cover ...,26,0.676068,"[cboneless, leg, of, lamb, olive, oil, fresh, ...",14532
134,20475,Zesty Citrus Salad,Jeff Kelly,2002-02-23T14:44:00Z,A very colorful salad that is beautiful to ser...,5.0,0.550000,This was a hit with my family... the dressing ...,cto make dressingmix orange juice vinegars hon...,20,0.675119,"[cwhite, wine, vinegar, raspberryflavored, whi...",13447
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4311,530924,Soma Sengupta Easter Recipes: Spring Pea Soup!...,Soma Sengupta,2017-03-29T22:42:00Z,What could be a better beginning to Easter din...,5.0,0.800000,This is a great recipe,cplace cream parsley and rosemary sprigs in a ...,20,0.675119,"[crosemary, sprigs, flat, leaf, parsley, heavy...",14221
4320,534898,Beer Cheese-Stuffed Pretzels,Food.com,2018-01-12T20:11:00Z,Make and share this Beer Cheese-Stuffed Pretze...,5.0,0.791667,"So delicious! I didn't stuff them, but I did d...",cfor the dough melt the butter in a small pot ...,26,0.676068,"[cunsalted, butter, warm, water, light, brown,...",17751
4322,535723,Sheet Pan Chicken Dinner,Dan Churchill,2018-04-30T21:21:00Z,Make and share this Sheet Pan Chicken Dinner r...,5.0,0.937500,This was beyond amazing!!,cmake selfsaucing chicken and vegetables prehe...,50,0.675840,"[cyellow, onion, garlic, red, bell, pepper, he...",13767
4328,537477,Mac &amp; Cheese Chile Relleno,Food.com,2018-11-12T18:48:00Z,Think of it as a mac n' cheese-stuffed pepper ...,5.0,1.000000,You must try this! Excellent!!! It rated a 10 ...,cpreheat oven to 425 degrees f line a baking s...,18,0.677847,"[cunsalted, butter, red, onions, garlic, clove...",19590


In [39]:
import numpy as np

# Create a condition for being a trend based on NormalizedRating and Number_of_Mentions
condition = (top_percentile_df['NormalizedRating'] >= top_percentile_df['NormalizedRating'].median()) & (top_percentile_df['Number_of_Mentions'] >= top_percentile_df['Number_of_Mentions'].median())

# Create a new column 'is_trend' based on the condition
top_percentile_df['is_trend'] = np.where(condition, 1, 0)


C:\Users\shira\AppData\Local\Temp\ipykernel_7612\3421759710.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_percentile_df['is_trend'] = np.where(condition, 1, 0)


In [40]:
top_percentile_df.to_csv('extract_trend.csv', index=False)
